In [1]:
import pandas as pd
from io import StringIO
import numpy as np
import os

In [ ]:
# Using readlines()
'''
### SLOW VERSION


file1 = open('./raw_db_competitions/2021/01-Training/01 - Floor 01 Indoors/T01_01.txt', 'r')
Lines = file1.readlines()

sensors_dict = {}

count = 0
# Strips the newline character
for line in Lines:
    count += 1
    if count > 13 and count != 27:
        if line[0] == '%' and count <= 29:
            start = line.find("'")
            end = max(len(line)-2,line[start+1:].find("'") + start + 1)
            line = line[start+1:end]
            if line[-1] == ";":
                line = line[:-1]
            sensors_dict[line[0:4]] = pd.read_csv(StringIO(line[5:]), sep=";")
        elif line[0] != '%' and line[0] != "\n":
            sensors_dict[line[0:4]] = pd.concat([sensors_dict[line[0:4]], 
                                                 pd.read_csv(StringIO(line[5:]), sep=";", header=None, names=sensors_dict[line[0:4]].columns)], axis=0)
            #print("Line{}: {}".format(count, line.strip()))        
    #assert count < 50

for key, value in sensors_dict.items():
    value.rest_index(drop=True, inplace=True)
'''

In [2]:
devices_df = pd.DataFrame([], columns=['UserTraj', 'device_id', 'device_manufacturer', 'device_model', 'notes', 'device_type'])
users_df = pd.DataFrame([], columns=['UserTraj', 'user_id', "username", "type", "notes"])
#places = pd.DataFrame([])

map_wifi_mac_to_code = {}
map_ble4_mac_to_code = {}
map_latlon_to_tile = {}

In [3]:
base_folder = "./raw_db_competitions/2021/01-Training/"
all_folders = np.asarray([f for f in os.listdir(base_folder) if f[0] != "."])
all_folders.sort()
prev = None
counter = 0
#folder_path = "./raw_db_competitions/2021/01-Training/01 - Floor 01 Indoors/"
for folder in all_folders:
    folder_path = base_folder+folder+"/"
    all_files = np.asarray([f for f in os.listdir(folder_path) if f[0] != "."])
    all_files.sort()
    #file_index = 0
    #prev = None
    #counter = 0

    for file_index in range(len(all_files)):
        print(all_files[file_index])
        file_name = folder_path+all_files[file_index]

        #newUser = prev != all_files[file_index][:all_files[file_index].find("_")]

        file = open(file_name, 'r')
        Lines = file.readlines()
        Lines = Lines[:np.where(np.asarray(Lines) == "\n")[0][0]]

        #if newUser:
        #users_df = pd.concat([users_df, pd.DataFrame([[all_files[file_index][:all_files[file_index].find(".")], all_files[file_index][:all_files[file_index].find("_")], "NULL", "trusted", "NULL"]], columns=users_df.columns)])
        #users_df["UserTraj"] = all_files[file_index][:all_files[file_index].find(".")]

        spec = Lines[np.where(np.asarray(Lines) == "%\n")[0][0]+1:np.where(np.asarray(Lines) == "%\n")[0][1]]
        if devices_df[(devices_df["device_manufacturer"] == spec[1][spec[1].find("\t")+1:spec[1].find("\n")]) & 
                      (devices_df["device_model"] ==spec[2][spec[2].find("\t")+1:spec[2].find("\n")])].empty:
            device_id = counter
            counter += 1
        else:
            device_id = devices_df[(devices_df["device_manufacturer"] == spec[1][spec[1].find("\t")+1:spec[1].find("\n")]) & 
                      (devices_df["device_model"] ==spec[2][spec[2].find("\t")+1:spec[2].find("\n")])].iloc[0]["device_id"]
        spec_list = [all_files[file_index][:all_files[file_index].find(".")],
                     device_id, 
                     spec[1][spec[1].find("\t")+1:spec[1].find("\n")],
                     spec[2][spec[2].find("\t")+1:spec[2].find("\n")],
                     "Android version " + spec[4][spec[4].find("\t")+1:spec[4].find("\n")] + "; API Android " + spec[3][spec[3].find("\t")+1:spec[3].find("\n")],
                     "smartphone"
                    ]
        devices_df = pd.concat([devices_df, pd.DataFrame([spec_list], columns=devices_df.columns)])
        users_df = pd.concat([users_df, pd.DataFrame([[all_files[file_index][:all_files[file_index].find(".")], spec_list[1], "NULL", "trusted", "NULL"]], columns=users_df.columns)])
        #users_df["UserTraj"] = all_files[file_index][:all_files[file_index].find(".")]
            #newUser = False

        if prev == None:
            Lines = Lines[np.where(np.asarray(Lines) == "%\n")[0][1]+2:]
            sensors_dict = {}
            for line in Lines:
                if line == "% \n":
                    break
                if "Eddystone" in line:
                    continue
                line = line[line.find("\t'")+2:line.find("'\n")]
                if line[-1] == ";":
                    line = line[:-1]
                sensors_dict[line[0:4]] = pd.read_csv(StringIO(line[5:].strip(" ")), sep=";")
                sensors_dict[line[0:4]]["UserTraj"] = None
        prev = all_files[file_index][:all_files[file_index].find("_")]

        file = open(file_name, 'r')
        Lines = file.readlines()

        Lines = Lines[np.where(np.asarray(Lines) == "\n")[0][0]+1:]
        for key, value in sensors_dict.items():
            value = value.loc[:, ~value.columns.str.contains('^Unnamed')]
            temp = np.char.split(np.char.strip(np.char.strip(np.asarray(Lines)[np.flatnonzero(np.core.defchararray.find(np.asarray(Lines),key)!=-1)], "\n"), key+";"), sep =';')
            temp = pd.DataFrame(pd.DataFrame(temp)[0].to_list(), columns=value.columns[:-1])
            temp["UserTraj"] = all_files[file_index][:all_files[file_index].find(".")]
            sensors_dict[key] = pd.concat([value, temp])


T01_01.txt
T01_02.txt
T01_03.txt
T01_04.txt
T01_05.txt
T01_06.txt
T01_07.txt
T01_08.txt
T01_09.txt
T01_10.txt
T02_01.txt
T02_02.txt
T02_03.txt
T02_04.txt
T02_05.txt
T02_06.txt
T02_07.txt
T02_08.txt
T02_09.txt
T02_10.txt
T03_01.txt
T03_02.txt
T03_03.txt
T03_04.txt
T03_05.txt
T03_06.txt
T03_07.txt
T03_08.txt
T03_09.txt
T03_10.txt
T04_01.txt
T04_02.txt
T04_03.txt
T04_04.txt
T04_05.txt
T04_06.txt
T04_07.txt
T04_08.txt
T04_09.txt
T04_10.txt
T05_01.txt
T05_02.txt
T05_03.txt
T05_04.txt
T05_05.txt
T05_06.txt
T05_07.txt
T05_08.txt
T05_09.txt
T05_10.txt
T06_01.txt
T06_02.txt
T06_03.txt
T06_04.txt
T06_05.txt
T06_06.txt
T06_07.txt
T06_08.txt
T06_09.txt
T06_10.txt
T07_01.txt
T07_02.txt
T07_03.txt
T07_04.txt
T07_05.txt
T07_06.txt
T07_07.txt
T07_08.txt
T07_09.txt
T07_10.txt
T08_01.txt
T08_02.txt
T08_03.txt
T08_04.txt
T08_05.txt
T08_06.txt
T08_07.txt
T08_08.txt
T08_09.txt
T08_10.txt
T09_01.txt
T09_02.txt
T09_03.txt
T09_04.txt
T09_05.txt
T09_06.txt
T09_07.txt
T09_08.txt
T09_09.txt
T09_10.txt
T10_01.txt

In [4]:
# CREATE THE MAP TO NATURAL IDENTIFIERS TO CODE

distinct_fp = np.unique(np.asarray(sensors_dict["POSI"][['Building ID(0,1,2..3)', 'floor ID(0,1,2..4)', 'Latitude(degrees)', ' Longitude(degrees)']], dtype=np.float64), axis=0)
map_latlon_to_tile = {}
for idx  in range(len(distinct_fp)):
    map_latlon_to_tile[tuple(distinct_fp[idx])] = idx

distinct_wifi_mac = np.unique(np.asarray(sensors_dict["WIFI"][['MAC_BSSID']], dtype=str), axis=0)
map_wifi_mac_to_code = {}
for idx in range(len(distinct_wifi_mac)):
    map_wifi_mac_to_code[distinct_wifi_mac[idx][0]] = idx

sensors_dict["BLE4"] = sensors_dict["BLE4"].dropna()
distinct_ble4_mac = np.unique(np.asarray(sensors_dict["BLE4"][['MAC']], dtype=str), axis=0)
map_ble4_mac_to_code = {}
for idx in range(len(distinct_ble4_mac)):
    map_ble4_mac_to_code[distinct_ble4_mac[idx][0]] = idx

In [5]:
# ARRANGE WIFI DATA IN DENSE MATRIX FORMAT, ORIGIANL SAMPLING
wifi_df = sensors_dict["WIFI"].replace({'MAC_BSSID':map_wifi_mac_to_code}).groupby(["UserTraj", "AppTimestamp(s)"])[['MAC_BSSID', "RSS(dBm)"]].agg(list).reset_index()

temp_wifi_np = np.full((len(wifi_df),len(map_wifi_mac_to_code)), np.nan)
for index, row in wifi_df.iterrows():
    temp_wifi_np[index][row["MAC_BSSID"]] = row["RSS(dBm)"]

wifi_df = pd.concat([wifi_df[["UserTraj", "AppTimestamp(s)"]], pd.DataFrame(temp_wifi_np,columns=list(map_wifi_mac_to_code.values()))], axis=1)
cols = list(wifi_df.columns)
cols.remove("UserTraj")
wifi_df[cols] = wifi_df[cols].astype(np.float64)
#wifi_df.loc[:, wifi_df.columns != "UserTraj"] = wifi_df.loc[:, wifi_df.columns != "UserTraj"].astype(np.float64)
wifi_df['fingerprint_id'] = np.nan
#wifi_df[["AppTimestamp(s)"]] = wifi_df[["AppTimestamp(s)"]].astype(np.float)


# ARRANGE BLE DATA IN DENSE MATRIX FORMAT, SAMPLING AT 1 SEC
sensors_dict["BLE4"]["AppTimestamp(s)"] = np.asarray(sensors_dict["BLE4"]["AppTimestamp(s)"], dtype=np.float).round() 
ble_df = sensors_dict["BLE4"].replace({'MAC':map_ble4_mac_to_code}).groupby(["UserTraj", "AppTimestamp(s)"])[['MAC', "RSSI(dBm)"]].agg(list).reset_index()

temp_ble_np = np.full((len(ble_df),len(map_ble4_mac_to_code)), np.nan)
for index, row in ble_df.iterrows():
    temp_ble_np[index][row["MAC"]] = row["RSSI(dBm)"]

ble_df = pd.concat([ble_df[["UserTraj", "AppTimestamp(s)"]], pd.DataFrame(temp_ble_np,columns=list(map_ble4_mac_to_code.values()))], axis=1)
cols = list(ble_df.columns)
cols.remove("UserTraj")
ble_df[cols] = ble_df[cols].astype(np.float64)
#ble_df.loc[:, ble_df.columns != "UserTraj"] = ble_df.loc[:, ble_df.columns != "UserTraj"].astype(np.float64)
ble_df['fingerprint_id'] = np.nan
#ble_df[["AppTimestamp(s)"]] = ble_df[["AppTimestamp(s)"]].astype(np.float)


# ARRANGE IMU DATA IN SEMI-CORRECT FORMAT
imu_df = sensors_dict["ACCE"][['UserTraj', "AppTimestamp(s)", "SensorTimestamp(s)", 'Acc_X(m/s^2)', 'Acc_Y(m/s^2)','Acc_Z(m/s^2)']].copy()
imu_df = imu_df.merge(sensors_dict["MAGN"][['Mag_X(uT)', 'Mag_Y(uT)', 'Mag_Z(uT)', "SensorTimestamp(s)"]], on="SensorTimestamp(s)", how="inner").merge(
    sensors_dict["GYRO"][['Gyr_X(rad/s)', 'Gyr_Y(rad/s)','Gyr_Z(rad/s)', "SensorTimestamp(s)"]], on="SensorTimestamp(s)", how="inner")
cols = list(imu_df.columns)
cols.remove("UserTraj")
imu_df[cols] = imu_df[cols].astype(np.float64)
#imu_df.loc[:, imu_df.columns != "UserTraj"] = imu_df.loc[:, imu_df.columns != "UserTraj"].astype(np.float64)
imu_df['fingerprint_id'] = np.nan
#imu_df[["AppTimestamp(s)"]] = imu_df[["AppTimestamp(s)"]].astype(np.float)


# ARRANGE GNSS DATA IN SEMI-CORRECT FORMAT
gnss_df = sensors_dict["GNSS"][['UserTraj', "AppTimestamp(s)", 'Latit(deg)', 'Long(deg)', 'Altitude(m)', 'SatInUse']].copy()
cols = list(gnss_df.columns)
cols.remove("UserTraj")
gnss_df[cols] = gnss_df[cols].astype(np.float64)
#gnss_df.loc[:, gnss_df.columns != "UserTraj"] = gnss_df.loc[:, gnss_df.columns != "UserTraj"].astype(np.float64)
gnss_df['fingerprint_id'] = np.nan
#gnss_df[["AppTimestamp(s)"]] = gnss_df[["AppTimestamp(s)"]].astype(np.float)


# ARRNAGE TESSELETION DATA IN CORRECT FORMAT
tile_df = pd.DataFrame(map_latlon_to_tile.keys(), columns = ["building", "floor", "coord_a_x", "coord_a_y"])
tile_df["tile"] = map_latlon_to_tile.values()
tile_df["tessellation_type"] = "logical" 
tile_df[["site", "coord_b_x", "coord_b_y", "coord_c_x", "coord_c_y", "coord_d_x", "coord_d_y"]] = "NULL"
tile_df = tile_df[["building", "floor", "site", "tile", "tessellation_type ",
                   "coord_a_x", "coord_a_y", "coord_b_x", "coord_b_y", "coord_c_x", "coord_c_y", "coord_d_x", "coord_d_y"]]


# ARRNAGE FINGERPRINT DATA IN CORRECT FORMAT
tile_df[["building", "floor"]] = tile_df[["building", "floor"]].astype(int)
fp_df = sensors_dict["POSI"].merge(tile_df.astype(str), 
                           left_on=["Latitude(degrees)", " Longitude(degrees)", "floor ID(0,1,2..4)", "Building ID(0,1,2..3)"], 
                           right_on=["coord_a_x", "coord_a_y", "floor", "building"]
                          )[['UserTraj','Timestamp(s)', 'Counter', 'Latitude(degrees)', ' Longitude(degrees)', "building", "floor", "site",	"tile"]].merge(devices_df[["UserTraj", "device_id"]]).merge(users_df[["UserTraj", "user_id"]])
fp_df['Counter'] = fp_df["Counter"].astype(int)
fp_df = fp_df.sort_values(by=["UserTraj", "Counter"])
fp_df[["preceded_by", "followed_by"]] = np.nan
fp_df.reset_index(drop=True, inplace=True)
for index, row in fp_df.copy().iterrows():
    if index > 0 and row["UserTraj"] == fp_df["UserTraj"][index-1]:
        fp_df.loc[index, "preceded_by"] = index-1
    if index < len(fp_df)-1 and row["UserTraj"] == fp_df["UserTraj"][index+1]:
        fp_df.loc[index, "followed_by"] = index+1
        
fp_df["Timestamp(s)"] = fp_df["Timestamp(s)"].astype(np.float64)  
for index, row in fp_df[::-1].iterrows():
    imu_df.loc[(imu_df["UserTraj"] == row["UserTraj"]) & (imu_df["AppTimestamp(s)"] <= row["Timestamp(s)"]), 'fingerprint_id'] = index
    wifi_df.loc[(wifi_df["UserTraj"] == row["UserTraj"]) & 
            (wifi_df["AppTimestamp(s)"] <= row["Timestamp(s)"] + 3) &
           (wifi_df["AppTimestamp(s)"] >= row["Timestamp(s)"] - 3), 'fingerprint_id'] = index
    ble_df.loc[(ble_df["UserTraj"] == row["UserTraj"]) & 
            (ble_df["AppTimestamp(s)"] <= row["Timestamp(s)"] + 3) &
           (ble_df["AppTimestamp(s)"] >= row["Timestamp(s)"] - 3), 'fingerprint_id'] = index
    gnss_df.loc[(gnss_df["UserTraj"] == row["UserTraj"]) & 
            (gnss_df["AppTimestamp(s)"] <= row["Timestamp(s)"] + 3) &
           (gnss_df["AppTimestamp(s)"] >= row["Timestamp(s)"] - 3), 'fingerprint_id'] = index

imu_df = imu_df[~pd.isna(imu_df['fingerprint_id'])]
wifi_df = wifi_df[~pd.isna(wifi_df['fingerprint_id'])]
ble_df = ble_df[~pd.isna(ble_df['fingerprint_id'])]
gnss_df = gnss_df[~pd.isna(gnss_df['fingerprint_id'])]

wifi_df['fingerprint_id'] = wifi_df['fingerprint_id'].astype(np.int)
ble_df['fingerprint_id'] = ble_df['fingerprint_id'].astype(np.int)
imu_df['fingerprint_id'] = imu_df['fingerprint_id'].astype(np.int)
gnss_df['fingerprint_id'] = gnss_df['fingerprint_id'].astype(np.int)

wifi_df = wifi_df.groupby("fingerprint_id").mean().reset_index()
ble_df = ble_df.groupby("fingerprint_id").mean().reset_index()
gnss_df = gnss_df.groupby("fingerprint_id").mean().reset_index()
gnss_df["SatInUse"] = gnss_df["SatInUse"].round()

    

In [6]:
# FINAL FORMATTING

# ARRANGE WIFI DATA IN CORRECT FORMAT
wifi_df = wifi_df.replace(np.nan, "NULL")
inv_map = {v: k for k, v in map_wifi_mac_to_code.items()}
wifi_df = wifi_df.drop("AppTimestamp(s)",axis=1)
new_cols = ["fingerprint_id"] + ["AP-WAP"+str(x)+"-"+ inv_map[x] for x in wifi_df.columns if x not in ("fingerprint_id", "AppTimestamp(s)")]
wifi_df = wifi_df.astype(str)
wifi_df.columns = new_cols


# ARRANGE BLE4 DATA IN CORRECT FORMAT
ble_df = ble_df.replace(np.nan, "NULL")
inv_map = {v: k for k, v in map_ble4_mac_to_code.items()}
ble_df = ble_df.drop("AppTimestamp(s)",axis=1)
new_cols = ["fingerprint_id"] + ["BL-BLE"+str(x)+"-"+ inv_map[x] for x in ble_df.columns if x not in ("fingerprint_id", "AppTimestamp(s)")]
ble_df = ble_df.astype(str)
ble_df.columns = new_cols


# ARRANGE IMU DATA IN CORRECT FORMAT
imu_df = imu_df[['SensorTimestamp(s)', 'Acc_X(m/s^2)',
       'Acc_Y(m/s^2)', 'Acc_Z(m/s^2)', 'Mag_X(uT)', 'Mag_Y(uT)', 'Mag_Z(uT)',
       'Gyr_X(rad/s)', 'Gyr_Y(rad/s)', 'Gyr_Z(rad/s)', 'fingerprint_id']]
imu_df.columns = ['epoch', 'acc_x', 'acc_y', 'acc_z', 'mag_x', 'mag_y', 'mag_z', 'gyr_x', 'gyr_y', 'gyr_z', 'fingerprint_id']
imu_df = imu_df[['fingerprint_id', 'epoch', 'acc_x', 'acc_y', 'acc_z', 'mag_x', 'mag_y', 'mag_z', 'gyr_x', 'gyr_y', 'gyr_z']]
imu_df = imu_df.astype(str)


# ARRANGE GNSS DATA IN CORRECT FORMAT
gnss_df['SatInUse'] = gnss_df['SatInUse'].replace(np.nan, -1).round().astype(int).replace(-1, "NULL")
gnss_df = gnss_df.replace(np.nan, "NULL")
gnss_df = gnss_df.drop("AppTimestamp(s)",axis=1)
gnss_df.columns = ['fingerprint_id', 'latitude', 'longitude', 'elevation', 'num_satellites']
gnss_df = gnss_df.astype(str)


# ARRANGE FINGERPRINT DATA IN CORRECT FORMAT
fp_df = fp_df.reset_index().drop(["Timestamp(s)", "UserTraj", "Counter"],axis=1)
fp_df[["preceded_by", "followed_by"]] = fp_df[["preceded_by", "followed_by"]].replace(np.nan, -1).astype(int).replace(-1, "NULL")
fp_df = fp_df.replace(np.nan, "NULL")
fp_df['set'] = "training"
fp_df['is_radio_map'] = True
fp_df['notes'] = "NULL"
fp_df['coord_z'] = "NULL"
fp_df['epoch'] = "NULL"
fp_df.columns = ["fingerprint_id", "coord_x", "coord_y", "building", "floor", "site", "tile", "device_id", 'user_id',
       'preceded_by', 'followed_by', 'set', 'is_radio_map', 'notes', 'coord_z',
       'epoch']
fp_df = fp_df[["fingerprint_id", "coord_x", "coord_y", 'coord_z', "building", "floor", "site", "tile", 'user_id', "device_id", 
       'epoch', 'set', 'is_radio_map', 'preceded_by', 'followed_by', 'notes']]
fp_df = fp_df.astype(str)


# ARRNAGE PLACES DATA IN CORRECT FORMAT
places_df = tile_df[["building", "floor", "floor"]].copy()
places_df.columns = ["building", "floor", "floor_number"]
places_df[["site", "site_height", "site_area", "floor_height", "floor_area", "building_area"]] = "NULL"
places_df = places_df.drop_duplicates()
places_df = places_df.astype(str)


# ARRANGE TESSELTATION DATA IN CORRECT FORMAT
tile_df = tile_df.astype(str)


# ARRANGE DEVICES DATA IN CORRECT FORMAT
devices_df = devices_df[["device_id", "device_model", "device_manufacturer", "device_type", "notes"]].drop_duplicates()
devices_df = devices_df.astype(str)
devices_df["device_manufacturer"] = devices_df["device_manufacturer"].str.capitalize()



# ARRANGE USERS DATA IN CORRECT FORMAT
users_df = users_df[["user_id", "username", "type", "notes"]].drop_duplicates()
users_df = users_df.astype(str)

In [24]:
display(wifi_df)
display(ble_df)
display(imu_df)
display(gnss_df)
display(fp_df)
display(places_df)
display(tile_df)
display(devices_df)
display(users_df)

,fingerprint_id,AP-WAP0-20:21:00:00:00:01,AP-WAP1-20:21:00:00:00:02,AP-WAP2-20:21:00:00:00:03,AP-WAP3-20:21:00:00:00:04,AP-WAP4-20:21:00:00:00:05,AP-WAP5-20:21:00:00:00:06,AP-WAP6-20:21:00:00:00:07,AP-WAP7-20:21:00:00:00:08,AP-WAP8-20:21:00:00:00:09,...,AP-WAP160-20:21:00:00:01:61,AP-WAP161-20:21:00:00:01:62,AP-WAP162-20:21:00:00:01:63,AP-WAP163-20:21:00:00:01:64,AP-WAP164-20:21:00:00:01:65,AP-WAP165-20:21:00:00:01:66,AP-WAP166-20:21:00:00:01:67,AP-WAP167-20:21:00:00:01:68,AP-WAP168-20:21:00:00:01:69,AP-WAP169-20:21:00:00:01:70
0,0,NULL,NULL,-64.0,-59.0,-71.0,-62.0,-72.0,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,NULL,-64.0,-59.0,-71.0,-62.0,-72.0,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,-64.0,-59.0,-71.0,-62.0,-72.0,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,-64.0,-59.0,-71.0,-62.0,-72.0,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,-70.0,-78.0,-78.0,-67.0,NULL,-45.0,-57.0,-78.0,-59.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,1092,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1073,1093,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1074,1094,NULL,NULL,NULL,-81.0,-71.0,NULL,NULL,NULL,-71.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1075,1097,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


,fingerprint_id,BLE4-BLE0-20:21:B0:00:00:01,BLE4-BLE1-20:21:B0:00:00:02,BLE4-BLE2-20:21:B0:00:00:03,BLE4-BLE3-20:21:B0:00:00:04,BLE4-BLE4-20:21:B0:00:00:05,BLE4-BLE5-20:21:B0:00:00:06,BLE4-BLE6-20:21:B0:00:00:07,BLE4-BLE7-20:21:B0:00:00:08,BLE4-BLE8-20:21:B0:00:00:09,...,BLE4-BLE1290-F4:75:42:F3:5A:9C,BLE4-BLE1291-F5:6D:8C:83:C1:BA,BLE4-BLE1292-F5:C8:B5:81:6A:09,BLE4-BLE1293-F5:FE:E0:E8:55:49,BLE4-BLE1294-F6:DF:39:B1:9F:A1,BLE4-BLE1295-F9:4E:E6:8E:73:4C,BLE4-BLE1296-FA:01:A7:EB:C4:5B,BLE4-BLE1297-FB:89:00:0C:0A:CF,BLE4-BLE1298-FB:C9:19:2A:07:C7,BLE4-BLE1299-FC:D9:35:9C:84:95
0,0,-92.25,-87.2,-90.0,-93.4,NULL,-67.6,-93.33333333333333,NULL,-95.8,...,NULL,NULL,NULL,-84.16666666666667,-94.33333333333333,-77.2,-94.0,NULL,NULL,NULL
1,1,-78.16666666666667,-77.33333333333333,-81.16666666666667,-81.16666666666667,-98.0,-68.33333333333333,-98.0,NULL,-94.0,...,NULL,NULL,NULL,-69.33333333333333,-89.16666666666667,-81.16666666666667,-94.5,NULL,NULL,NULL
2,2,-71.16666666666667,-69.4,-77.25,-83.6,NULL,-68.66666666666667,-91.25,NULL,-95.33333333333333,...,NULL,NULL,NULL,-91.5,-76.66666666666667,-93.33333333333333,-85.5,NULL,NULL,-91.0
3,3,-75.2,-73.66666666666667,-82.2,-83.16666666666667,NULL,-68.33333333333333,-88.5,NULL,-80.0,...,NULL,-95.66666666666667,NULL,-94.0,-85.8,-93.5,-78.4,NULL,NULL,-94.0
4,4,-90.0,-81.0,-93.0,-81.5,NULL,-69.0,-84.0,NULL,-85.33333333333333,...,NULL,-92.0,NULL,-89.33333333333333,-89.0,-97.0,-76.66666666666667,NULL,NULL,-90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1095,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,-89.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1096,1096,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,-90.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1097,1097,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,-81.66666666666667,-78.83333333333333,NULL,NULL,-87.16666666666667,NULL,-76.75,-79.0,-78.83333333333333,-65.5
1098,1098,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,-83.0,-77.5,NULL,NULL,-88.5,NULL,-90.0,-71.66666666666667,-74.16666666666667,-70.5


,fingerprint_id,epoch,acc_x,acc_y,acc_z,mag_x,mag_y,mag_z,gyr_x,gyr_y,gyr_z
0,0,NULL,-0.84518,2.31526,7.65929,-8.1,-12.9,-36.0,0.7216,-0.31945,-0.31393
1,0,NULL,-1.22108,2.65525,6.21075,-8.28,-13.02,-36.3,0.52306,-0.12519,-0.33776
2,0,NULL,-0.21309,3.03594,6.70158,-7.8,-13.08,-36.12,0.11928,0.30791,-0.2785
3,0,NULL,-1.16362,3.02157,7.10621,-6.96,-13.26,-36.12,-0.20448,0.30791,-0.39212
4,0,NULL,-0.44773,4.81489,7.779,-7.14,-13.26,-35.58,-0.33276,0.28409,-0.52896
...,...,...,...,...,...,...,...,...,...,...,...
1420386,1091,NULL,-0.13887,2.76299,9.83089,-28.2,-43.0,57.2,-0.31016,0.19606,0.04906
1420387,1091,NULL,-0.19872,2.83243,9.86202,-28.4,-42.9,57.2,-0.26435,0.37748,0.10587
1420388,1091,NULL,0.05746,2.96411,9.88596,-28.6,-42.8,57.2,-0.1715,0.38298,0.14497
1420389,1091,NULL,0.05507,3.10777,9.77822,-28.8,-42.7,57.2,0.00749,0.20339,0.11748


,fingerprint_id,latitude,longitude,elevation,num_satellites
0,28,38.886245,-7.007537,231.7,0
1,29,38.886195,-7.007515,231.6,0
2,35,38.886185,-7.007622,231.7,0
3,37,38.886187,-7.007629,231.7,0
4,39,38.886191,-7.007576,231.5,0
...,...,...,...,...,...
329,1059,38.886227,-7.0071125,222.78275000000002,8
330,1070,38.886226,-7.007317,231.8,0
331,1073,38.886226,-7.007317,231.285,0
332,1094,38.88624,-7.007335,231.8,0


,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,38.886312963794,-7.007964895876,NULL,80,1,NULL,48,0,0,NULL,training,True,NULL,1,NULL
1,1,38.886335004051,-7.007757070087,NULL,80,1,NULL,49,0,0,NULL,training,True,0,2,NULL
2,2,38.886205462399,-7.007736041601,NULL,80,1,NULL,22,0,0,NULL,training,True,1,3,NULL
3,3,38.886181560768,-7.007970377686,NULL,80,1,NULL,14,0,0,NULL,training,True,2,4,NULL
4,4,38.886184462061,-7.007938828219,NULL,80,1,NULL,15,0,0,NULL,training,True,3,5,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1095,38.886257188367,-7.007254264511,NULL,80,0,NULL,2,4,4,NULL,training,True,1094,1096,NULL
1096,1096,38.886257188367,-7.007254264511,NULL,80,0,NULL,2,4,4,NULL,training,True,1095,1097,NULL
1097,1097,38.886252306351,-7.007276099765,NULL,80,1,NULL,39,4,4,NULL,training,True,1096,1098,NULL
1098,1098,38.886252306351,-7.007276099765,NULL,80,1,NULL,39,4,4,NULL,training,True,1097,1099,NULL


,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,80,0,0,NULL,NULL,NULL,NULL,NULL,NULL
9,80,1,1,NULL,NULL,NULL,NULL,NULL,NULL


,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,80,0,NULL,0,logical,38.886225550993,-7.006846843305,NULL,NULL,NULL,NULL,NULL,NULL
1,80,0,NULL,1,logical,38.886230704147,-7.006796235068,NULL,NULL,NULL,NULL,NULL,NULL
2,80,0,NULL,2,logical,38.886257188367,-7.007254264511,NULL,NULL,NULL,NULL,NULL,NULL
3,80,0,NULL,3,logical,38.886278542427,-7.007018988466,NULL,NULL,NULL,NULL,NULL,NULL
4,80,0,NULL,4,logical,38.886281323786,-7.006856203057,NULL,NULL,NULL,NULL,NULL,NULL
5,80,0,NULL,5,logical,38.886286449973,-7.006805647101,NULL,NULL,NULL,NULL,NULL,NULL
6,80,0,NULL,6,logical,38.886297050993,-7.006819311975,NULL,NULL,NULL,NULL,NULL,NULL
7,80,0,NULL,7,logical,38.886314849441,-7.007025619961,NULL,NULL,NULL,NULL,NULL,NULL
8,80,0,NULL,8,logical,38.886334846136,-7.00682601002,NULL,NULL,NULL,NULL,NULL,NULL
9,80,1,NULL,9,logical,38.885730545313,-7.008075928819,NULL,NULL,NULL,NULL,NULL,NULL


,device_id,device_model,device_manufacturer,device_type,notes
0,0,SM-N960F,Samsung,smartphone,Android version 10; API Android 29
0,1,SM-N975F,Samsung,smartphone,Android version 11; API Android 30
0,2,SM-A520F,Samsung,smartphone,Android version 8.0.0; API Android 26
0,3,SM-G930F,Samsung,smartphone,Android version 8.0.0; API Android 26
0,4,Mi 10 Pro,Xiaomi,smartphone,Android version 10; API Android 29


,user_id,username,type,notes
0,0,NULL,trusted,NULL
0,1,NULL,trusted,NULL
0,2,NULL,trusted,NULL
0,3,NULL,trusted,NULL
0,4,NULL,trusted,NULL


In [9]:
# EXPORT DATA IN CSV
save_fodler = "./raw_db_competitions/2021/01-Training/"
wifi_df.to_csv(index=False, path_or_buf=save_fodler+"wifi_obs.csv")
ble_df.to_csv(index=False, path_or_buf=save_fodler+"blue_obs.csv")
imu_df.to_csv(index=False, path_or_buf=save_fodler+"imu_obs.csv")
gnss_df.to_csv(index=False, path_or_buf=save_fodler+"gnss_obs.csv")
fp_df.to_csv(index=False, path_or_buf=save_fodler+"fingerprints.csv")
places_df.to_csv(index=False, path_or_buf=save_fodler+"places.csv")
tile_df.to_csv(index=False, path_or_buf=save_fodler+"tessellations.csv")
devices_df.to_csv(index=False, path_or_buf=save_fodler+"devices.csv")
users_df.to_csv(index=False, path_or_buf=save_fodler+"users.csv")

In [23]:
imu_df

,fingerprint_id,epoch,acc_x,acc_y,acc_z,mag_x,mag_y,mag_z,gyr_x,gyr_y,gyr_z
0,0,NULL,-0.84518,2.31526,7.65929,-8.1,-12.9,-36.0,0.7216,-0.31945,-0.31393
1,0,NULL,-1.22108,2.65525,6.21075,-8.28,-13.02,-36.3,0.52306,-0.12519,-0.33776
2,0,NULL,-0.21309,3.03594,6.70158,-7.8,-13.08,-36.12,0.11928,0.30791,-0.2785
3,0,NULL,-1.16362,3.02157,7.10621,-6.96,-13.26,-36.12,-0.20448,0.30791,-0.39212
4,0,NULL,-0.44773,4.81489,7.779,-7.14,-13.26,-35.58,-0.33276,0.28409,-0.52896
...,...,...,...,...,...,...,...,...,...,...,...
1420386,1091,NULL,-0.13887,2.76299,9.83089,-28.2,-43.0,57.2,-0.31016,0.19606,0.04906
1420387,1091,NULL,-0.19872,2.83243,9.86202,-28.4,-42.9,57.2,-0.26435,0.37748,0.10587
1420388,1091,NULL,0.05746,2.96411,9.88596,-28.6,-42.8,57.2,-0.1715,0.38298,0.14497
1420389,1091,NULL,0.05507,3.10777,9.77822,-28.8,-42.7,57.2,0.00749,0.20339,0.11748
